In [34]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import scipy as sp
from scipy import stats
import statsmodels.stats.sandwich_covariance as sc

In [7]:
 df1 = pd.read_csv('FwdSpot1.dat', sep = '\s+', header = None, names = ['m', 'y', 'sgbp', 'sfrf', 'sjpy', 'f1gbp', 'f1frf', 'f1jpy'])

In [8]:
df2 = pd.read_csv('FwdSpot3.dat', sep = '\s+', header = None, names = ['m', 'y', 'sgbp', 'sfrf', 'sjpy', 'f3gbp', 'f3frf', 'f3jpy'])

In [9]:
for x in ['sgbp', 'sfrf', 'sjpy', 'f1gbp', 'f1frf', 'f1jpy']: df1['l' + x] = np.log(df1[x])
for x in ['sgbp', 'sfrf', 'sjpy', 'f3gbp', 'f3frf', 'f3jpy']: df2['l' + x] = np.log(df2[x])

In [12]:
df1df = df1[['l'+x for x in ['sgbp', 'sfrf', 'sjpy']]].diff(periods = 1, axis = 0) 
df1df = df1df.drop(df1df.index[0]).reset_index().drop(['index'], axis = 1)
df1df1 = pd.DataFrame()
for x in ['f1gbp', 'f1frf', 'f1jpy']:
    df1df1['l'+x] = df1['l'+x] - df1['ls'+x[2:]]
df1df1 = df1df1.drop(df1df1.index[-1]).reset_index().drop(['index'], axis = 1) 
for x in ['f1gbp', 'f1frf', 'f1jpy']:
    df1df['l'+x] = df1df1['l'+x]
df1df['ic'] = 1

In [13]:
df1df.head()

,lsgbp,lsfrf,lsjpy,lf1gbp,lf1frf,lf1jpy,ic
0,0.004595,-0.007289,0.002927,-0.002378,0.001583,0.007435,1
1,0.033647,0.052550,0.006622,-0.001810,0.000791,0.000797,1
2,0.004074,0.044546,0.004478,-0.001284,0.000434,0.003952,1
3,-0.029591,0.005792,-0.008711,-0.001701,0.000000,0.007593,1
4,-0.019615,-0.044714,-0.000796,-0.002386,0.001744,0.013430,1


In [14]:
mdp1 = sm.OLS(df1df['lsgbp'], df1df[['ic', 'lf1gbp']])
resp1 = mdp1.fit(cov_type = 'HC2')
mdf1 = sm.OLS(df1df['lsfrf'], df1df[['ic', 'lf1frf']])
resf1 = mdf1.fit(cov_type = 'HC2')
mdj1 = sm.OLS(df1df['lsjpy'], df1df[['ic', 'lf1jpy']])
resj1 = mdj1.fit(cov_type = 'HC2')

In [16]:
print(resp1.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsgbp   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.231
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.268
Time:                        13:53:18   Log-Likelihood:                 472.04
No. Observations:                 233   AIC:                            -940.1
Df Residuals:                     231   BIC:                            -933.2
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -0.0023      0.002     -0.962      0.3

In [17]:
print(resf1.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsfrf   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.232
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.268
Time:                        13:53:25   Log-Likelihood:                 469.21
No. Observations:                 233   AIC:                            -934.4
Df Residuals:                     231   BIC:                            -927.5
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -0.0023      0.003     -0.869      0.3

In [18]:
print(resj1.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsjpy   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.05814
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.810
Time:                        13:53:30   Log-Likelihood:                 466.52
No. Observations:                 233   AIC:                            -929.0
Df Residuals:                     231   BIC:                            -922.1
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic             0.0036      0.002      1.467      0.1

In [19]:
R = np.array([[1,0],[0,1]])
r = np.array([0,1])

In [21]:
print(resp1.wald_test(r_matrix = (R,r), use_f = False))

<Wald test (chi2): statistic=[[7.15623983]], p-value=0.027928156166202187, df_denom=2>


In [22]:
print(resf1.wald_test(r_matrix = (R,r), use_f = False))

<Wald test (chi2): statistic=[[5.52306321]], p-value=0.06319490443811177, df_denom=2>


In [23]:
print(resj1.wald_test(r_matrix = (R,r), use_f = False))

<Wald test (chi2): statistic=[[3.82192743]], p-value=0.14793774812787716, df_denom=2>


In [24]:
df2df = df2[['l'+x for x in ['sgbp', 'sfrf', 'sjpy']]].diff(periods = 3, axis = 0) 
df2df = df2df.drop(df2df.index[0:3]).reset_index().drop(['index'], axis = 1) 
df2df1 = pd.DataFrame()
for x in ['f3gbp', 'f3frf', 'f3jpy']:
    df2df1['l'+x] = df2['l'+x] - df2['ls'+x[2:]]
df3df1 = df2df1.drop(df2df1.index[-3:]).reset_index().drop(['index'], axis = 1) 
for x in ['f3gbp', 'f3frf', 'f3jpy']:
    df2df['l'+x] = df3df1['l'+x]
df2df['ic'] = 1

In [27]:
df2df.head()

,lsgbp,lsfrf,lsjpy,lf3gbp,lf3frf,lf3jpy,ic
0,0.042316,0.089807,0.014027,-0.006949,0.003855,0.023183,1
1,0.008130,0.102888,0.002389,-0.005239,0.002151,0.005036,1
2,-0.045132,0.005624,-0.005028,-0.003857,0.002050,0.014415,1
3,-0.067887,-0.023511,-0.010568,-0.004460,0.001074,0.021064,1
4,-0.029220,-0.022951,-0.040303,-0.007478,0.004585,0.038231,1


In [28]:
mdp2 = sm.OLS(df2df['lsgbp'], df2df[['ic', 'lf3gbp']])
resp2 = mdp2.fit(cov_type = 'HC2')
mdf2 = sm.OLS(df2df['lsfrf'], df2df[['ic', 'lf3frf']])
resf2 = mdf2.fit(cov_type = 'HC2')
mdj2 = sm.OLS(df2df['lsjpy'], df2df[['ic', 'lf3jpy']])
resj2 = mdj2.fit(cov_type = 'HC2')

In [36]:
print(resp2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsgbp   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     21.84
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           5.07e-06
Time:                        14:02:29   Log-Likelihood:                 333.32
No. Observations:                 231   AIC:                            -662.6
Df Residuals:                     229   BIC:                            -655.8
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -0.0187      0.005     -3.762      0.0

In [37]:
print(resf2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsfrf   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                    0.7880
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.376
Time:                        14:02:59   Log-Likelihood:                 323.12
No. Observations:                 231   AIC:                            -642.2
Df Residuals:                     229   BIC:                            -635.3
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -0.0044      0.005     -0.838      0.4

In [38]:
print(resj2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lsjpy   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     3.796
Date:                Wed, 01 Jun 2022   Prob (F-statistic):             0.0526
Time:                        14:03:27   Log-Likelihood:                 325.71
No. Observations:                 231   AIC:                            -647.4
Df Residuals:                     229   BIC:                            -640.5
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic             0.0131      0.004      3.161      0.0

In [32]:
R = R.reshape(2,2)
r = r.reshape(2,1)

In [33]:
np.array(resp2.params).reshape(2,1)

array([[-0.01871043],
       [-2.0586316 ]])

In [35]:
b2p = np.array(resp2.params).reshape(2,1)
b2f = np.array(resf2.params).reshape(2,1)
b2j = np.array(resj2.params).reshape(2,1)
NWvar2p = sc.cov_hac(resp2).reshape(2,2)
NWvar2f = sc.cov_hac(resf2).reshape(2,2)
NWvar2j = sc.cov_hac(resj2).reshape(2,2)
NWW2p = (R@b2p-r).T@np.linalg.inv(NWvar2p)@(R@b2p-r)
NWW2f = (R@b2f-r).T@np.linalg.inv(NWvar2f)@(R@b2f-r)
NWW2j = (R@b2j-r).T@np.linalg.inv(NWvar2j)@(R@b2j-r)
NWpval2p = 1 - stats.chi2.cdf(NWW2p, 2)
NWpval2f = 1 - stats.chi2.cdf(NWW2f, 2)
NWpval2j = 1 - stats.chi2.cdf(NWW2j, 2)
HHvar2p = sc.cov_hac(resp2, nlags = 2, weights_func = sc.weights_uniform).reshape(2,2)
HHvar2f = sc.cov_hac(resf2, nlags = 2, weights_func = sc.weights_uniform).reshape(2,2)
HHvar2j = sc.cov_hac(resj2, nlags = 2, weights_func = sc.weights_uniform).reshape(2,2)
HHW2p = (R@b2p-r).T@np.linalg.inv(HHvar2p)@(R@b2p-r)
HHW2f = (R@b2f-r).T@np.linalg.inv(HHvar2f)@(R@b2f-r)
HHW2j = (R@b2j-r).T@np.linalg.inv(HHvar2j)@(R@b2j-r)
HHpval2p = 1 - stats.chi2.cdf(HHW2p, 2)
HHpval2f = 1 - stats.chi2.cdf(HHW2f, 2)
HHpval2j = 1 - stats.chi2.cdf(HHW2j, 2)
print('Newey-West HAC Wald p-value for H0 for 3-per. pound is ', NWpval2p)
print('Newey-West HAC Wald p-value for H0 for 3-per. franc is ', NWpval2f)
print('Newey-West HAC Wald p-value for H0 for 3-per. yen is ', NWpval2j)
print('Hansen-Hodrick HAC Wald p-value for H0 for 3-per. pound is ', HHpval2p)
print('Hansen-Hodrick HAC Wald p-value for H0 for 3-per. franc009 is ', HHpval2f)
print('Hansen-Hodrick HAC Wald p-value for H0 for 3-per. yen is ', HHpval2j)

Newey-West HAC Wald p-value for H0 for 3-per. pound is  [[5.11342296e-05]]
Newey-West HAC Wald p-value for H0 for 3-per. franc is  [[0.14134594]]
Newey-West HAC Wald p-value for H0 for 3-per. yen is  [[0.00107895]]
Hansen-Hodrick HAC Wald p-value for H0 for 3-per. pound is  [[0.00024391]]
Hansen-Hodrick HAC Wald p-value for H0 for 3-per. franc009 is  [[0.18958511]]
Hansen-Hodrick HAC Wald p-value for H0 for 3-per. yen is  [[0.00144919]]


For **1-month** forward exchange rates there is no serial correlation. \
*For pound*: p-value = 0,028 -> H0 is rejected at 5% level. \
*For franc*: p-value = 0,063 -> H0 is not rejected at 5% level. \
*For yen*: p-value = 0,148 -> H0 is not rejected at 10% level. \
For **3-month** forward exchange rates there is serial correlation. Newey-West and Hansen-Hodrick are applied. \
 *For pound*: p-value = 0,0002 -> H0 is rejected at 1% level. \
*For franc*: p-value = 0,189 -> H0 is not rejected at 10% level. \
*For yen*: p-value = 0,001 -> H0 is rejected at 1% level. \
Thus, current forward rates are not the best predictors of the spot rates. It works only for some currencies for different periods, i.e. it works for franc for 1&3 month periods; for yen for 1-month period; does not work for pounds. 
